## Import and Read

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import os
import anndata
import omicverse as ov
import matplotlib as mpl
import decoupler as dc
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
ov.plot_set(font_path='Arial')

# Enable auto-reload for development
%load_ext autoreload
%autoreload 2

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['text.usetex'] = False

# sc.settings.verbosity = 1  
# sc.settings.set_figure_params(dpi=80, figsize=(3, 3), facecolor="white")  
os.getcwd()

In [ ]:
os.chdir('..')
adata = sc.read_h5ad('./semla_output/ctb_st_clean.h5ad')
adata

In [ ]:
adata.layers["norm"] = adata.X.copy()
adata

## Spatial Weighting

In [ ]:
sample_adatas = []

for sample_id in adata.obs['sample'].unique():
    sample_adata = adata[adata.obs['sample'] == sample_id].copy()
    dc.pp.knn(sample_adata, key="spatial", bw=100, cutoff=0.1)
    
    # Update X with spatially weighted gene exression
    sample_adata.X = sample_adata.obsp["spatial_connectivities"].dot(sample_adata.X)  # dot矩阵相乘
    sample_adatas.append(sample_adata)

sample_adatas

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(15,10))
fig.suptitle('Weighting Genes',fontsize=16, y=0.98)
axes = axes.flatten()
for i,sample in enumerate(sample_adatas):
    sc.pl.spatial(sample, color='SPP1', show=False, ax=axes[i], spot_size=50, cmap="Reds", )

plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()
plt.close()

In [ ]:
from matplotlib import rc_context
fig, axes = plt.subplots(2,3, figsize=(15,10))
fig.suptitle('No Weighting Genes',fontsize=16, y=0.98)
axes = axes.flatten()
with rc_context({'figure.figsize':(5, 5)}):
    for i, sample in enumerate(adata.obs['sample'].unique()):
        sc.pl.spatial(adata[adata.obs['sample']==sample, ], color='SPP1', show=False, ax=axes[i], spot_size=50, cmap="Reds", )

    plt.tight_layout()
    plt.show()

## Enrichment Analysis

### TF_Regulons 

In [ ]:
adata

In [ ]:
# collectri = dc.op.collectri(organism="human")
collectri = pd.read_csv('semla_output/CollecTRI_regulons.csv')
collectri

In [ ]:
collectri['source'].value_counts().index

In [ ]:
dc.mt.ulm(data=adata, net=collectri)

In [ ]:
score = dc.pp.get_obsm(adata=adata, key="score_ulm")
score

In [ ]:
# score.write_h5ad('semla_output/adata_tf_score.h5ad')

In [ ]:
score=sc.read_h5ad('semla_output/adata_tf_score.h5ad')
score

In [ ]:
len(adata.var.index.intersection(score.var.index).to_numpy())

In [ ]:
df = dc.tl.rankby_group(adata=score, groupby="first_type", reference="rest", method="t-test_overestim_var")
df = df[(df['padj']<0.05) & (df["stat"] > 0.0)]
df

In [ ]:
adata.var.index.dtype, score.var.index.dtype

In [ ]:
len(df['name'].value_counts().index)

In [ ]:
common_genes = adata.var.index.astype(str).intersection(score.var.index.astype(str))
df['name'] = df['name'].astype(str)
df = df[df['name'].isin(common_genes)]
df

In [ ]:
df[df['group']=="TREM2 Mac"].head(20)

In [ ]:
len(df['name'].value_counts().index)

In [ ]:
n_markers = 3
source_markers = (
    df.groupby("group")
    .head(n_markers)
    # .drop_duplicates("name")
    .groupby("group")["name"]
    .apply(lambda x: list(x))
    .to_dict()
)
source_markers
df[df['group']=="SPP1 Mac"].head(20)

In [ ]:
# from pyrasterize import rasterize
tf = "HIF1A"
fig, axes = plt.subplots(2,3, figsize=(15,10))
fig.suptitle('HIF1A score',fontsize=16, y=0.98)
axes = axes.flatten()
with rc_context({'figure.figsize':(5, 5), 'figure.dpi':300}):
    for i, sample in enumerate(score.obs['sample'].unique()):
        sc.pl.spatial(score[score.obs['sample']==sample, ], color=tf, show=False, ax=axes[i], spot_size=50, cmap="RdBu_r", vmax=10, vmin=-2, title=f'{sample}',)
    plt.tight_layout()
    plt.show()

In [ ]:
# from pyrasterize import rasterize
tf = "HIF1A"
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('HIF1A score', fontsize=16, y=0.98)
axes = axes.flatten()
save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight','pad_inches': 0.5}
with rc_context({'figure.dpi': 300}):
    for i, sample in enumerate(score.obs['sample'].unique()):
        sc.pl.spatial(
            score[score.obs['sample'] == sample, ], color=tf, show=False, ax=axes[i], spot_size=50, cmap="RdBu_r", vmax=10, vmin=-2, title=f'{sample}')
        # for collection in axes[i].collections:
        #     if hasattr(collection, 'set_rasterized'):
        #         collection.set_rasterized(True)
        # axes[i].collections[0].set_rasterized(True)
        # axes[i].set_rasterized(True)
    plt.tight_layout()
    fig.savefig(f"./python_figures/HIF1A_spatial_rasterized.{save_params['format']}", **save_params)
    plt.show()
    plt.close()

In [ ]:
from plot_fun.spatial_utils import plot_feature_expression
fig = plot_feature_expression(adata, 'HIF1A',vmax=2.5, vmin=0,)
# fig.savefig('./python_figures/HIF1A_exp_spatial.pdf', dpi=300, bbox_inches='tight', transparent=False)  
plt.show()

In [ ]:
sc.pl.matrixplot(
    adata=score,
    var_names=source_markers,
    groupby="first_type",
    dendrogram=True,
    standard_scale="var",
    colorbar_title="Z-scaled scores",
    cmap="RdBu_r",
)

### PROGENy Pathways

In [ ]:
# progeny = dc.op.progeny(organism="human", top=500)
# progeny

In [ ]:
# progeny.to_csv('semla_output/progeny.csv')
progeny=pd.read_csv('semla_output/progeny.csv', index_col=0)
progeny

In [ ]:
progeny.to_csv("./data/progeny.csv",index=False)

In [ ]:
progeny['source'].value_counts().index

In [ ]:
progeny[progeny['source']=='Hypoxia'].head(20)

In [ ]:
dc.mt.ulm(data=adata, net=progeny)

In [ ]:
score_pro = dc.pp.get_obsm(adata=adata, key="score_ulm")
score_pro

In [ ]:
# score_pro.write_h5ad('semla_output/adata_progeny_score.h5ad')
score_pro = sc.read_h5ad('semla_output/adata_progeny_score.h5ad')
score_pro

In [ ]:
from matplotlib import rc_context
save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight','pad_inches': 0.5}
pw = "Hypoxia"
fig, axes = plt.subplots(2,3, figsize=(15,10))
fig.suptitle('Hypoxia score',fontsize=16, y=0.98)
axes = axes.flatten()
with rc_context({'figure.figsize':(5, 5)}):
    for i, sample in enumerate(score_pro.obs['sample'].unique()):
        sc.pl.spatial(score_pro[score_pro.obs['sample']==sample, ], color=pw, show=False, ax=axes[i], spot_size=50, cmap="RdBu_r", vmax=10, vmin=-2, title=f"{sample}",)
    plt.tight_layout()
    fig.savefig(f"./python_figures/Hypoxia_progeny_rasterized.{save_params['format']}", **save_params)
    plt.show()
    plt.close()

In [ ]:
save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight','pad_inches': 0.5}
pw = "Hypoxia"
fig, axes = plt.subplots(2,3, figsize=(15,10))
fig.suptitle('Hypoxia score',fontsize=16, y=0.98)
axes = axes.flatten()
with rc_context({'figure.figsize':(5, 5)}):
    for i, sample in enumerate(score_pro.obs['sample'].unique()):
        sc.pl.spatial(score_pro[score_pro.obs['sample']==sample, ], color=pw, show=False, ax=axes[i], spot_size=50, cmap="RdBu_r", vmax=10, vmin=-2, title=f"{sample}",)
        # axes[i].set_rasterized(True)
    plt.tight_layout()
    fig.savefig(f"./python_figures/HIF1A_progeny_rasterized.{save_params['format']}", **save_params)
    plt.show()
    plt.close()

In [ ]:
subtype_cols=["#E6194B", "#3CB44B", "#4363D8", "#FFE119", "#F58231", "#911EB4", "#46F0F0", "#F032E6",
             "#BCBD22", "#008080", "#E6BEFF", "#9A6324", "#FFFAC8", "#AAFFC3", "#808000", "#FFD8B1",
             "#000075", "#A9A9A9", "#1F77B4", "#FF7F0E", "#2CA02C", "#D62728", "#9467BD", "#8C564B",
             "#E377C2", "#7F7F7F", "#17BECF", "#FF9898", "#C5B0D5", "#C49C94", "#F7B6D2", "#C7C7C7",
             "#DBDB8D", "#9EDAE5", "#FFBB78", "#98DF8A", "#AEC7E8", "#FF9E9E", "#B5D0D9", "#D4B9D6",
             "#FDB462", "#80B1D3"]
# fig,ax=plt.subplots(1,1,figsize=(15,6))
# sc.pl.violin(score_pro, keys=[pw], groupby="first_type", rotation=90, ax=ax, jitter=False)
from matplotlib import rc_context
save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight','pad_inches': 0.5,
              'metadata': {'Creator': None, 'Producer': None, 'CreationDate': None},}
pw="Hypoxia"
fig,ax=plt.subplots(1,1,figsize=(12,4))
# sc.pl.violin(score, keys=[pw], groupby="subtype", rotation=90, ax=ax, jitter=False)
with rc_context({'figure.figsize':(12,4)}):
    sc.pl.violin(score_pro, keys=[pw], groupby="first_type", rotation=90, ax=ax, jitter=False, palette=subtype_cols, stripplot=False,)
    plt.tight_layout()
    # fig.set_rasterized(True)
    fig.savefig(f"./python_figures/Hypoxia_score_st.{save_params['format']}", **save_params)
    plt.show()
    plt.close()

In [ ]:
score_pro

In [ ]:
save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight',}
fig, ax, stats = plot_simple_beautified_violin(score_pro, pw="Hypoxia")
fig.savefig("./python_figures/Hypoxia.pdf", dpi=300, bbox_inches='tight', transparent=False)
plt.show()
plt.close()

In [ ]:
sc.pl.matrixplot(
    adata=score_pro,
    var_names=score_pro.var_names,
    groupby="first_type",
    dendrogram=True,
    standard_scale="var",
    colorbar_title="Z-scaled scores",
    cmap="RdBu_r",
)

### Hallmark gene sets

In [ ]:
hallmark = dc.op.hallmark(organism="human")
hallmark

In [ ]:
dc.mt.ulm(data=adata, net=hallmark)

In [ ]:
adata

In [ ]:
adata.obsm['score_ulm'].shape

In [ ]:
score_hallmark = dc.pp.get_obsm(adata=adata, key="score_ulm")
score_hallmark

In [ ]:
# score_hallmark.write_h5ad('semla_output/adata_hallmark_score.h5ad')
score_hallmark = sc.read_h5ad('semla_output/adata_hallmark_score.h5ad')
score_hallmark

In [ ]:
df_hallmark = dc.tl.rankby_group(adata=score_hallmark, groupby="first_type", reference="rest", method="t-test_overestim_var")
df_hallmark

In [ ]:
df_hallmark['name'].value_counts().index

In [ ]:
df_hallmark = df_hallmark[(df_hallmark["stat"] > 0) & (df_hallmark["padj"] < 0.05) ]
df_hallmark 

In [ ]:
subtype_cols=["#E6194B", "#3CB44B", "#4363D8", "#FFE119", "#F58231", "#911EB4", "#46F0F0", "#F032E6",
             "#BCBD22", "#008080", "#E6BEFF", "#9A6324", "#FFFAC8", "#AAFFC3", "#808000", "#FFD8B1",
             "#000075", "#A9A9A9", "#1F77B4", "#FF7F0E", "#2CA02C", "#D62728", "#9467BD", "#8C564B",
             "#E377C2", "#7F7F7F", "#17BECF", "#FF9898", "#C5B0D5", "#C49C94", "#F7B6D2", "#C7C7C7",
             "#DBDB8D", "#9EDAE5", "#FFBB78", "#98DF8A", "#AEC7E8", "#FF9E9E", "#B5D0D9", "#D4B9D6",
             "#FDB462", "#80B1D3"]

from matplotlib import rc_context
import matplotlib
save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight','pad_inches': 0.5,
              'metadata': {'Creator': None, 'Producer': None, 'CreationDate': None},}
pw="INTERFERON_GAMMA_RESPONSE"
fig,ax=plt.subplots(1,1,figsize=(12,4))
# with rc_context({'figure.figsize':(12,4)}):
sc.pl.violin(score_hallmark, keys=[pw], groupby="first_type", rotation=90, ax=ax, jitter=False, palette=subtype_cols, stripplot=False,)
plt.tight_layout()
# fig.set_rasterized(True)
for ax in fig.get_axes():
    for collection in ax.collections:
        if isinstance(collection, matplotlib.collections.PathCollection):
            collection.set_rasterized(True)
# fig.savefig(f"./python_figures/INTERFERON_GAMMA_RESPONSE_score_st.{save_params['format']}", **save_params)
fig.savefig(
    f"./python_figures/INTERFERON_GAMMA_RESPONSE_score_st.pdf",
    dpi=300,
    bbox_inches='tight',
    transparent=False
)
plt.show()
plt.close()

In [ ]:
score_hallmark.var_names

In [ ]:
score_hallmark[:, pw].X.flatten()

In [ ]:
data = pd.DataFrame({
    'first_type': score_hallmark.obs['first_type'],
    'score': score_hallmark[:, pw].X.flatten()
}, index=score_hallmark.obs.index)
data

In [ ]:
cell_types = ['Basal KC', 'Inflammatory KC', 'Proliferating KC',
              'Spinous KC', 'Granular KC', 'Hair follicle', 'Sweet Gland',
              'Schwann Cell', 'Melanocyte', 'CCL19+ FB', 'APCDD1+ FB',
              'PI16+ FB', 'POSTN+ FB', 'TNN+ FB', 'RAMP1+ FB',
              'Arterial EC', 'Capillary EC', 'Venular EC1', 'Venular EC2',
              'Lymphatic Endothelial', 'Smooth muscle', 'Mast cell',
              'Langerhans', 'cDC1', 'cDC2A', 'cDC2B', 'pDC', 'M1-like Mac',
              'M2-like Mac', 'SPP1 Mac', 'TREM2 Mac', 'CD8T', 'Treg',
              'CD4 Naïve', 'Th17', 'CD8T_NK', 'Th17_Treg', 'NK', 'naïve B',
              'activated B', 'memory B', 'Plasma']
subtype_cols=["#E6194B", "#3CB44B", "#4363D8", "#FFE119", "#F58231", "#911EB4", "#46F0F0", "#F032E6",
             "#BCBD22", "#008080", "#E6BEFF", "#9A6324", "#FFFAC8", "#AAFFC3", "#808000", "#FFD8B1",
             "#000075", "#A9A9A9", "#1F77B4", "#FF7F0E", "#2CA02C", "#D62728", "#9467BD", "#8C564B",
             "#E377C2", "#7F7F7F", "#17BECF", "#FF9898", "#C5B0D5", "#C49C94", "#F7B6D2", "#C7C7C7",
             "#DBDB8D", "#9EDAE5", "#FFBB78", "#98DF8A", "#AEC7E8", "#FF9E9E", "#B5D0D9", "#D4B9D6",
             "#FDB462", "#80B1D3"]
color_dict = dict(zip(cell_types, subtype_cols))
color_dict

In [ ]:
import seaborn as sns
def plot_simple_beautified_violin(score_hallmark, pw="INTERFERON_GAMMA_RESPONSE", figsize=(14, 6)):
    if pw in score_hallmark.var_names:
        data = pd.DataFrame({
            'first_type': score_hallmark.obs['first_type'],
            'score': score_hallmark[:, pw].X.flatten()}, index=score_hallmark.obs.index)
    else:
        data = score_hallmark.obs[['first_type', pw]].copy()
        data.columns = ['first_type', 'score']
    
    data = data.dropna(subset=['score'])
    stats = data.groupby('first_type')['score'].agg(['mean', 'median']).reset_index()
    
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    # cell_types = data['first_type'].unique()
    # colors = subtype_cols[:len(cell_types)]
    color_dicts = color_dict
    
    sns.violinplot(data=data, x='first_type', y='score', palette=color_dicts,
                   ax=ax, inner=None, saturation=0.8)
    
    for i, cell_type in enumerate(cell_types):
        median_val = stats[stats['first_type'] == cell_type]['median'].values[0]
        # ax.hlines(median_val, i-0.25, i+0.25, color='white', linewidth=6, 
        #          zorder=10, label='Median' if i == 0 else "")
        ax.hlines(median_val, i-0.25, i+0.25, color='black', linewidth=3, 
                 zorder=11, label='Median' if i == 0 else "")
    
    ax.set_title(f'{pw} - Beautified Violin Plot', fontsize=16, fontweight='bold', pad=20)
    ax.set_ylabel('Response Score', fontsize=12, fontweight='bold')
    ax.set_xlabel('Cell Types', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=90, labelsize=9)
    
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.tick_params(axis='both', colors='black')
    
    # ax.legend(loc='upper right')
    plt.tight_layout()
    return fig, ax, stats

save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight',}
fig, ax, stats = plot_simple_beautified_violin(score_hallmark)
fig.savefig("./python_figures/INTERFERON_GAMMA_RESPONSE.pdf", dpi=300, bbox_inches='tight', transparent=False)
plt.show()
plt.close()

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_simple_beautified_violin(score_hallmark, pw="INTERFERON_GAMMA_RESPONSE", figsize=(14, 6)):
    if pw in score_hallmark.var_names:
        data = pd.DataFrame({
            'first_type': score_hallmark.obs['first_type'],
            'score': score_hallmark[:, pw].X.flatten()}, index=score_hallmark.obs.index)
    else:
        data = score_hallmark.obs[['first_type', pw]].copy()
        data.columns = ['first_type', 'score']
    
    data = data.dropna(subset=['score'])
    stats = data.groupby('first_type')['score'].agg(['mean', 'median']).reset_index()
    
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    color_dicts = color_dict
    
    sns.violinplot(data=data, x='first_type', y='score', palette=color_dicts,
                   ax=ax, inner=None, saturation=0.8,  linewidth=0, )
    
    for i, cell_type in enumerate(cell_types):
        median_val = stats[stats['first_type'] == cell_type]['median'].values[0]
        ax.hlines(median_val, i-0.25, i+0.25, color='white', linewidth=6, 
                 zorder=10, label='Median' if i == 0 else "")
        ax.hlines(median_val, i-0.25, i+0.25, color='black', linewidth=3, 
                 zorder=11, label='Median' if i == 0 else "")
    
    y_min = data['score'].min() - 2 
    y_max = data['score'].max() + 2  
    ax.set_ylim(y_min, y_max)
    ax.set_xlim(-0.8, len(cell_types) + 0.2)
    
    ax.set_title(f'{pw} - Beautified Violin Plot', fontsize=16, fontweight='bold', pad=20)
    ax.set_ylabel('Response Score', fontsize=12, fontweight='bold')
    ax.set_xlabel('Cell Types', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=90, labelsize=9)
    
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.tick_params(axis='both', colors='black')
    
    plt.tight_layout()
    return fig, ax, stats

save_params = {'dpi': 300, 'format': 'pdf', 'bbox_inches': 'tight'}
fig, ax, stats = plot_simple_beautified_violin(score_hallmark)
fig.savefig("./python_figures/INTERFERON_GAMMA_RESPONSE.pdf", **save_params)
plt.show()
plt.close()

In [ ]:
save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight',}
fig, ax, stats = plot_simple_beautified_violin(score_pro, pw="Hypoxia")
# fig.savefig("./python_figures/Hypoxia.pdf", dpi=300, bbox_inches='tight', transparent=False)
plt.show()
plt.close()

In [ ]:
score_hallmark.obs['orig.ident'].value_counts()

In [ ]:
import seaborn as sns
def plot_simple_beautified_violin(score_hallmark, pw="INTERFERON_GAMMA_RESPONSE", figsize=(14, 6)):
    if pw in score_hallmark.var_names:
        data = pd.DataFrame({
            'first_type': score_hallmark.obs['first_type'],
            'score': score_hallmark[:, pw].X.flatten()}, index=score_hallmark.obs.index)
    else:
        data = score_hallmark.obs[['first_type', pw]].copy()
        data.columns = ['first_type', 'score']
    
    stats = data.groupby('first_type')['score'].agg(['mean', 'median']).reset_index()
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    cell_types = data['first_type'].unique()
    colors = subtype_cols[:len(cell_types)]
    color_dict = dict(zip(cell_types, colors))
    
    sns.violinplot(data=data, x='first_type', y='score', palette=color_dict,
                   ax=ax, inner=None, saturation=0.8)
    sns.boxplot(data=data, x='first_type', y='score', 
                width=0.3, boxprops={'facecolor':'none'}, showfliers=False)
    
    ax.set_title(f'{pw} - Beautified Violin Plot', fontsize=16, fontweight='bold', pad=20)
    ax.set_ylabel('Response Score', fontsize=12, fontweight='bold')
    ax.set_xlabel('Cell Types', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=90, labelsize=9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    return fig, ax, stats

fig, ax, stats = plot_simple_beautified_violin(score_hallmark)
# fig.savefig(f"./python_figures/INTERFERON_GAMMA_RESPONSE_.pdf", **save_params)
plt.show()
plt.close()

In [ ]:
import seaborn as sns
def plot_simple_beautified_violin(score_hallmark, pw="INTERFERON_GAMMA_RESPONSE", figsize=(14, 6)):
    if pw in score_hallmark.var_names:
        data = pd.DataFrame({
            'first_type': score_hallmark.obs['first_type'],
            'score': score_hallmark[:, pw].X.flatten()}, index=score_hallmark.obs.index)
    else:
        data = score_hallmark.obs[['first_type', pw]].copy()
        data.columns = ['first_type', 'score']
    
    stats = data.groupby('first_type')['score'].agg(['mean', 'median']).reset_index()
    
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    cell_types = data['first_type'].unique()
    colors = subtype_cols[:len(cell_types)]
    color_dict = dict(zip(cell_types, colors))
    
    sns.violinplot(data=data, x='first_type', y='score', palette=color_dict,
                   ax=ax, inner=None, saturation=0.8, linewidth=0)  
    
    sns.boxplot(data=data, x='first_type', y='score', 
                width=0.5,  
                boxprops={'facecolor':'none'}, 
                medianprops={'color': 'black', 'linewidth': 2},
                showfliers=False)
    
    ax.set_title(f'{pw} - Beautified Violin Plot', fontsize=16, fontweight='bold', pad=20)
    ax.set_ylabel('Response Score', fontsize=12, fontweight='bold')
    ax.set_xlabel('Cell Types', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=90, labelsize=9)
    
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.tick_params(axis='both', colors='black')
    
    plt.tight_layout()
    return fig, ax, stats

fig, ax, stats = plot_simple_beautified_violin(score_hallmark)
#fig.savefig(f"./python_figures/INTERFERON_GAMMA_RESPONSE_.{save_params['format']}", **save_params)
plt.show()
plt.close()

In [ ]:
df_hallmark[df_hallmark['group']=='TREM2 Mac'].head(30)

In [ ]:
df_hallmark[df_hallmark['group']=='SPP1 Mac'].head(30)

In [ ]:
n_markers = 3
source_markers = (
    df_hallmark.groupby("group")
    .head(n_markers)
    # .drop_duplicates("name")
    .groupby("group")["name"]
    .apply(lambda x: list(x))
    .to_dict()
)
source_markers

In [ ]:
hms = ["INTERFERON_GAMMA_RESPONSE", ]
fig, axes = plt.subplots(2,3, figsize=(15,10))
fig.suptitle('INTERFERON_GAMMA_RESPONSE score',fontsize=16, y=0.98)
axes = axes.flatten()
with rc_context({'figure.figsize':(5, 5)}):
    for i, sample in enumerate(score_hallmark.obs['sample'].unique()):
        sc.pl.spatial(score_hallmark[score_hallmark.obs['sample']==sample, ], color=hms, show=False, 
                      ax=axes[i], spot_size=50, cmap="RdBu_r", vmax=18, vmin=0, title=f"{sample}", )
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)  
    fig.savefig(f"./python_figures/INTERFERON_GAMMA_RESPONSE_HM_rasterized.{save_params['format']}", **save_params)
    plt.show()
    plt.close()

In [ ]:
hms = ["FATTY_ACID_METABOLISM", ]
fig, axes = plt.subplots(2,3, figsize=(15,10))
fig.suptitle('FATTY_ACID_METABOLISM score',fontsize=16, y=0.98)
axes = axes.flatten()
with rc_context({'figure.figsize':(5, 5)}):
    for i, sample in enumerate(score_hallmark.obs['sample'].unique()):
        sc.pl.spatial(score_hallmark[score_hallmark.obs['sample']==sample, ], color=hms, show=False, 
                      ax=axes[i], spot_size=50, cmap="RdBu_r", vmax=6, vmin=0, title=f"{sample}", )
    plt.tight_layout()
    plt.subplots_adjust(top=0.92) 
    fig.savefig(f"./python_figures/FATTY_ACID_METABOLISM_HM_rasterized.{save_params['format']}", **save_params)
    plt.show()
    plt.close()